# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import plotly.express as px

# Import API key
from api_keys import geoapify_key

c:\Users\jaypa\AppData\Local\Programs\Python\Python39\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (None)/charset_normalizer (3.3.2) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("C:/Users/jaypa/OneDrive/Documents/GitHub/UofT Bootcamp/python_api_challenge/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,buncrana,55.1333,-7.4500,277.03,72,100,3.16,IE,1707263542
1,1,port-aux-francais,-49.3500,70.2167,279.53,92,100,10.05,TF,1707263543
2,2,zouerate,22.7187,-12.4521,291.35,10,12,4.34,MR,1707263543
3,3,ribeira grande,38.5167,-28.7000,290.80,94,90,6.71,PT,1707263543
4,4,bodo,67.2800,14.4050,267.17,86,100,8.23,NO,1707263543


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

fig = px.scatter_geo(city_data_df,
                     lat='Lat',
                     lon='Lng',
                     color='Humidity',  # Color of markers can be set to humidity
                     size='Humidity',  # Size of markers can also reflect humidity
                     hover_name='City',  # Hover text can be set to city name
                     projection='natural earth',  # Map projection style
                     title='Cities Humidity Levels')  # Title of the map

# Show the map
fig.show()

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Drop any rows with null values
city_data_df.dropna(inplace=True)

# Apply the criteria
filtered_cities_df = city_data_df[
    (city_data_df['Max Temp'] > 294.15) & (city_data_df['Max Temp'] < 300.15) & 
    (city_data_df['Wind Speed'] < 4.5) & 
    (city_data_df['Cloudiness'] == 0)
]

# Display a sample of the data
filtered_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
73,73,saint-pierre,-21.3393,55.4781,296.97,60,0,3.09,RE,1707263554
103,103,narasannapeta,18.4167,84.0500,295.02,87,0,1.75,IN,1707263559
217,217,nova sintra,14.8667,-24.7167,294.32,71,0,4.32,CV,1707263577
219,219,ratnagiri,16.9833,73.3000,297.68,60,0,3.53,IN,1707263578
265,265,talcahuano,-36.7167,-73.1167,294.97,78,0,3.09,CL,1707263584
268,268,macas,-2.3167,-78.1167,299.01,69,0,1.54,EC,1707263308
346,346,punta del este,-34.9667,-54.9500,298.21,83,0,3.60,UY,1707263599
378,378,vila velha,-20.3297,-40.2925,297.12,92,0,1.34,BR,1707263604
388,388,mansoa,12.0733,-15.3189,297.26,73,0,0.46,GW,1707263605
406,406,xai-xai,-25.0519,33.6442,295.21,81,0,0.74,MZ,1707263608


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# 1. Create hotel_df DataFrame
hotel_df = filtered_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# 2. Add an empty column for Hotel Name
hotel_df['Hotel Name'] = ''

# 3. Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
73,saint-pierre,RE,-21.3393,55.4781,60,
103,narasannapeta,IN,18.4167,84.0500,87,
217,nova sintra,CV,14.8667,-24.7167,71,
219,ratnagiri,IN,16.9833,73.3000,60,
265,talcahuano,CL,-36.7167,-73.1167,78,
268,macas,EC,-2.3167,-78.1167,69,
346,punta del este,UY,-34.9667,-54.9500,83,
378,vila velha,BR,-20.3297,-40.2925,92,
388,mansoa,GW,12.0733,-15.3189,73,
406,xai-xai,MZ,-25.0519,33.6442,81,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000  # Search within 10,000 meters (10 km) of the city coordinates
params = {
    'apiKey': geoapify_key,
    'categories': 'accommodation.hotel',  # Use the correct category for hotels
    'limit': 1  # We only need the first result
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat, lng = row['Lat'], row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Check for successful response
    if response.status_code == 200 and 'features' in name_address and len(name_address['features']) > 0:
        # Grab the first hotel from the results and store the name
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        print(f"Hotel found for {row['City']}: {hotel_df.loc[index, 'Hotel Name']}")
    else:
        # If no hotel is found, or if there's an error, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        print(f"No hotel found or error for {row['City']}: {response.status_code}, {name_address}")

# Display sample data
hotel_df.head()



Starting hotel search
Hotel found for saint-pierre: Tropic Hotel
No hotel found or error for narasannapeta: 200, {'type': 'FeatureCollection', 'features': []}
Hotel found for nova sintra: Residência Ka Dencho
Hotel found for ratnagiri: Vihar
Hotel found for talcahuano: Club Recreativo Armada de Chile
Hotel found for macas: Nivel 5
Hotel found for punta del este: Hotel Aquarium
Hotel found for vila velha: Hotel Prainha
Hotel found for mansoa: Uaque Resort Hotel
No hotel found or error for xai-xai: 200, {'type': 'FeatureCollection', 'features': []}


,City,Country,Lat,Lng,Humidity,Hotel Name
73,saint-pierre,RE,-21.3393,55.4781,60,Tropic Hotel
103,narasannapeta,IN,18.4167,84.0500,87,No hotel found
217,nova sintra,CV,14.8667,-24.7167,71,Residência Ka Dencho
219,ratnagiri,IN,16.9833,73.3000,60,Vihar
265,talcahuano,CL,-36.7167,-73.1167,78,Club Recreativo Armada de Chile


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Create the map
fig = px.scatter_geo(hotel_df,
                     lat='Lat',
                     lon='Lng',
                     hover_name='City',
                     hover_data={'Country': True, 'Hotel Name': True, 'Humidity': True, 'Lat': True, 'Lng': True},
                     projection='natural earth',
                     title='Cities with Hotel and Environmental Information')

# Show the map
fig.show()